<a href="https://colab.research.google.com/github/Seogury/cp2_team3/blob/main/%EC%B6%94%EC%B2%9C_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###추천 시스템(recommend algorithm)

In [ ]:
import pandas as pd 

#필요한 데이터 불러오기
ele_item = pd.read_csv('/content/gdrive/MyDrive/ele_item.csv')
app_item = pd.read_csv('/content/gdrive/MyDrive/app_item.csv')
com_item = pd.read_csv('/content/gdrive/MyDrive/com_item.csv')

In [ ]:
#index 설정
ele_item.set_index('product_id', inplace = True)
app_item.set_index('product_id', inplace = True)
com_item.set_index('product_id', inplace = True)

In [ ]:
#유저 정보, product 정보
product = pd.read_csv('/content/gdrive/MyDrive/product.csv')
# user_pur = pd.read_csv('/content/gdrive/MyDrive/user_pur.csv') (이거는 리스트 데이터라 불러오면 str로 변환됨 )
#index 설정
product.set_index('product_id_2', inplace = True)

In [ ]:
user_pur.reset_index(inplace = True)

In [ ]:
product = product[['product_id', 'product_pur_count', 'brand', 'price',  'main_category','middle_category','sub_category']]

In [ ]:
com_item.head(5)

,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
product_id,,,,,,,,,,,,,,,,,,,,,
1303049,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1303355,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304106,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304297,0.0,0.0,0.0,1.0,0.0,0.0,0.146071,0.178073,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304346,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def search(id):
  x = user_pur[user_pur['user_id']== id]
  return x

def recent_item(id):
  x = user_pur[user_pur['user_id']==id]['product'].values[0][-1]
  return x

def recent_category(id):
  x = user_pur[user_pur['user_id']==id]['main_category'].values[0][-1]
  return x

def recent_brand(id):
  x = user_pur[user_pur['user_id']==id]['brand'].values[0][-1]
  return x

def recent_middle(id):
  x = user_pur[user_pur['user_id']==id]['middle'].values[0][-1]
  return x

def id_category(df):
  #가장 많은 이벤트가 있는 카테고리
  cat = df[['ele_count',	'app_count',	'com_count'	]]
  cat = cat.T
  cat.reset_index(inplace= True)
  index = cat.columns[1]
  max = cat[index].max()
  if max == 50: #최빈값 2개 이상일때
    max_cat = recent_category(id)
  else:
    max_cat = cat.loc[cat[index] == cat[index].max()]['index'].item()
  

  if max_cat == 'ele_count' or max_cat == 'electronics':
    category = 'electronics'
  elif max_cat == 'app_count'or max_cat == 'appliances':
    category = 'appliances'
  else:
    category = 'computers'


  return category

In [ ]:
def ele_recommend(data):  

  #전자제품일 경우
  ele = data[['apple_count', 'samsung_count', 'other_count']]
  ele = ele.T
  ele.reset_index(inplace= True)
  index = ele.columns[1]
  max = ele[index].max()
  if max == 50: #최빈값 2개 이상
    max_brand = recent_brand(id)
  else:
    max_brand = ele.loc[ele[index] == ele[index].max()]['index'].item()

  if max_brand == 'apple_count' or max_brand == 'apple':
    brand = 'apple'
  elif max_brand == 'samsung_count' or max_brand == 'samsung':
    brand = 'samsung'
  else:
    brand = 'other'



  #중분류
  middle = data['middle']

  try:
    must_middle = statistics.mode(list(data['middle'])[0]) 
  except: #최빈값 2개 이상일때
    must_middle = recent_middle(id)

  #가격대
  price = data['mean_price']
  user_price = price.item()
  high_price = user_price + 100
  low_price = user_price - 100 
  

  #브랜드관 추천
    
  if brand == 'apple':
    #product 구분 > product_id 추출
    recommend10 = product[(product['brand']== 'apple')&
                        (product['main_category'] == 'electronics')].sort_values('product_pur_count', ascending = False).head(11)
    recommend = product[(product['brand']== 'apple')&
                        (product['main_category'] == 'electronics')].sort_values('product_pur_count', ascending = False)

  elif brand == 'samsung':
    #product 구분
    recommend10 = product[(product['brand']== 'samsung')&
                        (product['main_category'] == 'electronics')].sort_values('product_pur_count', ascending = False).head(11)
    recommend = product[(product['brand']== 'samsung')&
                        (product['main_category'] == 'electronics')].sort_values('product_pur_count', ascending = False)

  else :
    recommend10 = product[(product['brand']!= 'samsung')&
                        (product['brand']!= 'apple')&
                        (product['main_category']=='electronics')&
                        (product['middle_category'] == must_middle)&
                        (product['price'] <= high_price) & 
                        (product['price'] >= low_price) ].sort_values('product_pur_count', ascending = False).head(11)
    recommend = product[(product['brand']!= 'samsung')&
                        (product['brand']!= 'apple')&
                        (product['main_category']=='electronics')&
                        (product['middle_category'] == must_middle)&
                        (product['price'] <= high_price) & 
                        (product['price'] >= low_price) ].sort_values('product_pur_count', ascending = False)

  return recommend10[['product_id','brand', 'price', 'main_category', 'middle_category','sub_category' ]], recommend[['product_id','brand', 'price', 'main_category', 'middle_category','sub_category' ]]

In [ ]:
import statistics

def app_recommend(data):  
  
  #가전 제품일 경우

  x = data['middle']
  
  try:
    must_middle = statistics.mode(list(data['middle'])[0]) 
  except: #최빈값 2개 이상일때
    must_middle = recent_middle(id)

  product[product['middle_category'] == must_middle]

  #가전 추천
  recommend10 = product[product['middle_category'] == must_middle].sort_values('product_pur_count', ascending = False).head(10)
  recommend = product[product['middle_category'] == must_middle].sort_values('product_pur_count', ascending = False)
  
  return recommend10[['product_id','brand', 'price', 'main_category', 'middle_category','sub_category' ]], recommend[['product_id','brand', 'price', 'main_category', 'middle_category' ,'sub_category']]

In [ ]:
def com_recommend(data): 

 #가격대 검색  
  x = data['mean_price'] 
  user_price = x.item() 
  high_price = user_price + 100 
  low_price = user_price - 100 

  #중분류
  middle = data['middle']
  m_list = []
  for i in middle:
    m_list.append(i)
  
  
  try:
    must_middle = statistics.mode(list(data['middle'])[0])  
  except: #최빈값 2개 이상일때
    must_middle = recent_middle(id)

  
  #brand
  brand = data['brand']

  try:
    must_brand = statistics.mode(list(data['brand'])[0])   
  except:
    must_brand = recent_brand(id)
  
  #조건부 탐색
  if must_brand == 'apple':
    recommend10 = product[(product['price'] <= high_price) & 
                        (product['price'] >= low_price) & 
                        (product['middle_category'] == must_middle)].sort_values('product_pur_count', ascending = False).head(11)
    recommend = product[(product['price'] <= high_price) & 
                        (product['price'] >= low_price) & 
                        (product['middle_category'] == must_middle)].sort_values('product_pur_count', ascending = False)     
  else:
    recommend10 = product[(product['price'] <= high_price) &
                        (product['price'] >= low_price) &
                        (product['middle_category'] == must_middle)].sort_values('product_pur_count', ascending = False).head(11)
    recommend = product[(product['price'] <= high_price) &
                        (product['price'] >= low_price) &
                        (product['middle_category'] == must_middle)].sort_values('product_pur_count', ascending = False)

  return recommend10[['product_id','brand', 'price', 'main_category', 'middle_category','sub_category' ]], recommend[['product_id','brand', 'price', 'main_category', 'middle_category','sub_category' ]]

In [ ]:
def item_recommend_CF(df, pro, id, top = 11):

  # 조건부 내 유사도 탐색
  inx = df.T[id].sort_values(ascending = False).index
  inx = list(inx)
  inx = list(map(int,inx))
  result = product.loc[inx]

  return result[['brand','price','main_category','middle_category','sub_category']]

In [ ]:
def search_col(df):
  index = df['product_id'].values
  col = list(index)
  col = list(map(str, col))
  return col

In [ ]:
def id_recommend(id):

  data = search(id)
  recent = recent_item(id)
  cat = id_category(data)


  if cat == 'electronics':
    recommend10, recommend = ele_recommend(data)
    col = search_col(recommend)
    try:
        CF_recommend = item_recommend_CF(ele_item[col], recommend, recent, top = 11)
    except:
        try:
            CF_recommend = item_recommend_CF(app_item[col], recommend, recent, top = 11)
        except:
            CF_recommend = item_recommend_CF(com_item[col], recommend, recent, top = 11)
  
  elif cat == 'appliances':
    recommend10, recommend = app_recommend(data)
    col = search_col(recommend)
    try:
        CF_recommend = item_recommend_CF(app_item[col], recommend, recent, top = 11)
    except:
        try:
            CF_recommend = item_recommend_CF(ele_item[col], recommend, recent, top = 11)
        except:
            CF_recommend = item_recommend_CF(com_item[col], recommend, recent, top = 11)
  
  else:
    recommend10, recommend = com_recommend(data)
    col = search_col(recommend)
    try:
        CF_recommend = item_recommend_CF(com_item[col], recommend, recent, top = 11)
    except:
        try:
            CF_recommend = item_recommend_CF(ele_item[col], recommend, recent, top = 11)
        except:
            CF_recommend = item_recommend_CF(app_item[col], recommend, recent, top = 11)
  

  return recommend10, CF_recommend.head(11)

In [ ]:
pop,cf = id_recommend(566265448)

In [ ]:
cf

,brand,price,main_category,middle_category,sub_category
product_id_2,,,,,
1600587,epson,205.41,computers,peripherals,printer
1500458,epson,164.16,computers,peripherals,printer
1600593,epson,174.52,computers,peripherals,printer
1600558,canon,192.49,computers,peripherals,printer
1600557,canon,167.21,computers,peripherals,printer
1600585,epson,205.64,computers,peripherals,printer
1600554,hp,118.82,computers,peripherals,printer
1600596,hp,136.16,computers,peripherals,printer
1600002,canon,275.17,computers,peripherals,printer


####precision

In [ ]:
test_user = com_user.sort_values(418115333, ascending = False).head(3000).index

In [ ]:
for i in test_user:
  print(i)

In [ ]:
user1.reset_index(inplace = True)

In [ ]:
pop_test = []
cf_test = []

for i in test_user:
  try:
    #true positive
    pur = user1[user1['user_id'] == i]['product'].item()
    pur= list(set(pur))
    pop,cf = id_recommend(i)
    cf_count = 0
    for i in pur:      
      for j in cf.index[1:]: #예측
         if i==j:
           cf_count += 1
           rate =  cf_count/len(cf.index)
           cf_test.append(rate)

    pop_count = 0
    for i in pur:      
      for j in pop.index: #예측
        if i == j:
          pop_count += 1
          rate =  pop_count/len(pop.index)
          pop_test.append(rate)
  except:
    cf_test.append('err')
    pop_test.append('err')

In [ ]:
#에러 삭제
for item in pop_test:
    if item == 'err':
        pop_test.remove(item)

In [ ]:
#에러 삭제
for item in cf_test:
    if item == 'err':
        cf_test.remove(item)

In [ ]:
len(pop_test)

4327

In [ ]:
print('pop_algorithm_precision = ',np.mean(pop_test))

pop_algorithm_precision =  0.20450911271918898


In [ ]:
print('cf_algorithm_precision = ',np.mean(cf_test))

cf_algorithm_precision =  0.17082594029347276
